In [17]:
import spacy
nlp = spacy.load('es_core_news_md')


In [ ]:
from keras.models import Model, load_model

In [ ]:
model = load_model('40_0.5_0.25_200_3_0.0105_Nadam.h5')

In [ ]:
import numpy as np

In [ ]:
from preprocessing import readfile, createBatches, createMatrices, iterate_minibatches, addCharInformation, padding

In [ ]:
charList =['(', 'q', '·', 'O', ')', '?', ':', 'Y', 'é', 'x', 'D', 'y', 'P', 'H', 'W', '9', 'I', 'm', 'l', 'N', 'o', '"', 'J', '6', 'à', 't', 'Ñ', 'Q', 'ñ', 'K', 'n', '=', 'k', '3', '¡', '2', 'ú', 'R', 'p', 's', 'G', '1', 'M', 'g', 'z', '@', 'h', 'r', 'A', 'j', 'E', 'C', '0', '-', "'", 'e', 'w', '+', 'V', '7', 'v', 'u', 'B', 'a', '¿', '.', 'f', 'U', 'í', 'ó', '4', '%', 'á', 'S', 'd', 'c', '&', 'ü', '8', 'L', '!', '5', '/', 'X', 'T', 'Z', ';', ',', 'F', 'i', 'b']

In [ ]:
class EvaluateRealInput(object):
    def __init__(self, INPUT, MODEL, charList):
        self.input = nlp(INPUT)
        self.Model = MODEL
        self.charList = charList
        
    def load_data(self):
        
        sentences = []
        sentence = []
        for sent in self.input.sents:
            #print(sent)
            sentence = []
            for token in sent:
                #print(token)
                if token.text == '\n' or token.text == '\n\n'  :
                    continue
                else:
                    sentence.append([token.text, ''])
            sentences.append(sentence)
        """for token in self.input:
            if len(token.text) ==0 or token.text == "\n":
                if len(sentence) > 0:
                    sentences.append(sentence)
                    sentence = []
                continue
            sentence.append([token.text.replace(' ', ''), ''])
        if len(sentence) > 0:
            sentences.append(sentence)
            sentence = []"""
        
        self.sentences = sentences
    
    def addCharInfo(self):
        self.sentences = addCharInformation(self.sentences)
        
    def Embed(self):
        labelSet = set()
        words = {}
        for sentence in self.sentences:
            for token, char, label in sentence:
                # token ... token, char ... list of chars, label ... BIO labels   
                labelSet.add(label)
                #charSet.add(char)
                words[token.lower()] = True
        
        print(words)
        self.label2Idx = {}
        for label in labelSet:
            self.label2Idx[label] = len(self.label2Idx)
                
        case2Idx = {'numeric': 0, 'allLower': 1, 'allUpper': 2, 'initialUpper': 3, 'other': 4, 'mainly_numeric': 5,
                    'contains_digit': 6, 'PADDING_TOKEN': 7}
        
        self.caseEmbeddings = np.identity(len(case2Idx), dtype='float32')
        
        word2Idx = {}
        self.wordEmbeddings = []

        fEmbeddings = open("./datasets/Embeddings/eswiki_20180420_100d.txt", encoding="utf-8")
        fEmbeddings.readline()

        # loop through each word in embeddings
        for line in fEmbeddings:
            split = line.strip().split(" ")
            word = split[0]  # embedding word entry

            if len(word2Idx) == 0:  # add padding+unknown
                word2Idx["PADDING_TOKEN"] = len(word2Idx)
                vector = np.zeros(len(split) - 1)  # zero vector for 'PADDING' word
                self.wordEmbeddings.append(vector)

                word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
                vector = np.random.uniform(-0.25, 0.25, len(split) - 1)
                self.wordEmbeddings.append(vector)

            if split[0].lower() in words:
                vector = np.array([float(num) for num in split[1:]])
                self.wordEmbeddings.append(vector)  # word embedding vector
                word2Idx[split[0]] = len(word2Idx)  # corresponding word dict

        self.wordEmbeddings = np.array(self.wordEmbeddings)
        self.char2Idx = {"PADDING": 0, "UNKNOWN": 1}
        #charList=list(charSet)
        for c in self.charList:
            self.char2Idx[c] = len(self.char2Idx)
            
        self.input_set = padding(createMatrices(self.sentences, word2Idx, self.label2Idx, case2Idx, self.char2Idx))
        self.idx2Label = {v: k for k, v in self.label2Idx.items()}
        self.idx2Word = {v: k for k, v in word2Idx.items()}
    
    def createBatches(self):
        """Create batches"""
        self.input_batch, self.input_batch_len = createBatches(self.input_set)
    
    def tag_input(self):
        """Tag data with numerical values"""
        correctLabels = []
        predLabels = []
        for i, data in enumerate(dataset):
            tokens, casing, char, labels = self.input_set[i]
            tokens = np.asarray([tokens])
            casing = np.asarray([casing])
            char = np.asarray([char])
            pred = self.Model.predict([tokens, casing, char], verbose=False)[0]
            pred = pred.argmax(axis=-1)  # Predict the classes
            print(pred)
        #return predLabels, correctLabels

In [ ]:
string = """La contracampaña electoral de Trump se basa en desarrollar una total operación de descrédito que cale entre los votantes y que ya fue utilizada contra Hillary Clinton en 2016, cuando se puso en duda la salud física de la aspirante demócrata a la Casa Blanca. Antiguos asesores de la ex secretaria de Estado ya han advertido al equipo de Biden que necesitan tomarse el asunto con la máxima seriedad. "Biden no está respondiendo a las agresiones verbales y necesita hacerlo porque estos chismes calan en la opinión pública", explica Philippe Reines, principal consejero de Clinton en 2016."""

In [15]:
string = "Sao Paulo (Brasil), 23 may (EFECOM)."

In [18]:
doc = nlp(string)

In [21]:
for token in doc:
    if token.is_punct:
        print('OK', token)

OK (
OK )
OK ,
OK (
OK )
OK .


In [ ]:
evaluate = EvaluateRealInput(string, model, charList)
evaluate.load_data()
evaluate.addCharInfo()

In [ ]:
evaluate.sentences

In [ ]:
evaluate.Embed()

In [ ]:
evaluate.createBatches()

In [ ]:
tokens, casing, char, labels = evaluate.input_set[0]
tokens = np.asarray([tokens])
casing = np.asarray([casing])
char = np.asarray([char])

In [ ]:
pred = evaluate.Model.predict([tokens, casing, char], verbose=False)[0]


In [ ]:
pred = pred.argmax(axis=-1)

In [ ]:
print(pred)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle('idx2label.pkl','gzip')

In [ ]:
evaluate.idx2Word

In [ ]:
idx2label  = df.to_dict()['label']

In [ ]:
def showResults(tokens, pred):
    result = []
    for i,token in enumerate(tokens[0]):
        #print(i)
        #print(idx2label)
        result.append((evaluate.idx2Word[token], idx2label[pred[i]]))
    return result

In [ ]:
showResults(tokens, pred)

In [ ]:
#string = "HOLA MUNDO"

In [ ]:
TokenGetter(sent)

In [1]:
fEmbeddings = open("./datasets/Embeddings/eswiki_20180420_100d.txt", encoding="utf-8")

In [2]:
data = fEmbeddings.readlines()

In [3]:
data[0]

'1828809 100\n'

In [4]:
data = data[1:]

In [8]:
words = [array.split(" ")[0] for array in data]